# Multi Location, Multiple Temporal Scales, Multiple Operations Including Transport, Mixed Integer Linear Programming Example

## [Example 4]

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming with Material and Emission Considerations Example' [Example 3]. 

We update Example 3 to now allow multiple locations and Transport options between them.


## From Example 3

In [ ]:
from energia import *

m = Model('example4')
m.q = Period()
m.y = 4 * m.q
m.usd = Currency()


## Declaring Locations 

In [ ]:
m.ho = Location(label = 'Houston')
m.sd = Location(label = 'San Diego')
m.ny = Location(label = 'New York') 
# m.usa = m.ho + m.sd + m.ny

### The Network 

Given that we have not used a nested set of locations, a network is made:

In [3]:
m.network

ntw

This network contains the three locations at (ho, sd, ny)

In [ ]:
m.network.has 

(sd, ho, ny)

## General Bounds Default to Network

In [ ]:
m.declare(Resource, ['power', 'wind', 'solar'])
# m.solar.consume == True 
# m.wind.consume == True
m.solar.consume <= 1200
m.wind.consume <= 1200
m.consume.show()

--- General Resource Balance for solar in (ntw, y): initializing constraint, adding consume
    Completed in 0.0007498264312744141 seconds
--- General Resource Balance for wind in (ntw, y): initializing constraint, adding consume
    Completed in 0.00029206275939941406 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Location-specific Bounds 

Spatial specificity can be provided along with temporal fidelity. 
In the example below, the temporal fidelity is implied since the data has 4 samples.

Note that an individual general resource balance is generated for power at each location

In [6]:
m.power.release(m.ho).prep(30) >= [0.6, 0.7, 0.8, 0.3]
m.power.release(m.sd).prep(100) >= [0.4, 0.9, 0.7, 0.6]
m.power.release(m.ny).prep(180) >= [0.2, 0.5, 0.7, 0.5]
m.release.show()

--- General Resource Balance for power in (ho, q): initializing constraint, adding release
    Completed in 0.0002617835998535156 seconds
--- General Resource Balance for power in (sd, q): initializing constraint, adding release
    Completed in 0.0002579689025878906 seconds
--- General Resource Balance for power in (ny, q): initializing constraint, adding release
    Completed in 0.00023603439331054688 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## For All Locations 

Generally, in Energia to repeat a constraint over a set, use .forall()

If a list is provided, the bounds are iterated over the list [see LB cons]
else the bounds are repeated [see UB cons]

NameError: name 'Process' is not defined

In [ ]:
m.wf = Process(m.power)
m.wf(m.power) == -1 * m.wind
[10, 7, 20] <= m.wf.capacity.x.forall(m.network.has)
m.wf.capacity.x.forall(m.network.has) <= 100
m.wf.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Bounds can also be provided individually

In [ ]:
m.wf.operate.prep(norm = True)(m.ho) <= [0.9, 0.8, 0.5, 0.7]
m.wf.operate.prep(norm = True)(m.sd) <= [0.8, 0.7, 0.6, 0.5]
m.wf.operate.prep(norm = True)(m.ny) <= [0.6, 0.5, 0.4, 0.3]
m.operate.show(True)
# m.wf.operate.prep(norm = True).forall(m.network.has) <= [[0.9, 0.8, 0.5, 0.7], [0.8, 0.7, 0.6, 0.5], [0.6, 0.5, 0.4, 0.3]]

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Calculations can also be provided for all locations as a list or repeated

In [ ]:
m.wf.capacity[m.usd.spend].forall(m.network.has) == [90000, 70000, 60000]
m.wf.operate[m.usd.spend] == 49

--- Creating map to (usd, ntw, y, operate, wf). Mapping spend: from (usd, ho, y, operate, wf) to (usd, ntw, y, operate, wf)
    Completed in 0.0009720325469970703 seconds
--- Mapping spend: from (usd, sd, y, operate, wf) to (usd, ntw, y, operate, wf)
    Completed in 0.0005941390991210938 seconds
--- Mapping spend: from (usd, ny, y, operate, wf) to (usd, ntw, y, operate, wf)
    Completed in 0.0002338886260986328 seconds
--- Mapping operate across time from (wf, ho, q) to (wf, ho, y)
--- Creating map to (wf, ho, y). Mapping operate: from (wf, ho, q) to (wf, ho, y)
    Completed in 0.0001850128173828125 seconds
--- Creating map to (wf, ntw, y). Mapping operate: from (wf, ho, y) to (wf, ntw, y)
    Completed in 0.00048279762268066406 seconds
--- Mapping operate across time from (wf, sd, q) to (wf, sd, y)
--- Creating map to (wf, sd, y). Mapping operate: from (wf, sd, q) to (wf, sd, y)
    Completed in 0.0007290840148925781 seconds
--- Mapping operate: from (wf, sd, y) to (wf, ntw, y)
   

In [14]:
m.wf.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In the case where location specific data is not provided, the parameter is assumed tuo apply to the network

In [15]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar

In [ ]:

m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm = True).forall(m.network.has) <= [[0.6, 0.8, 0.9, 0.7], 
                                                           [0.5, 0.7, 0.6, 0.5], 
                                                           [0.4, 0.6, 0.5, 0.4]]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000   

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

m.gwp = Environ()

m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000

m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17 
m.cement.consume[m.gwp.emit] == 0.9

m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- Aspect (capacity) not defined at sd, a variable will be created assuming y as the temporal index
--- Creating map to (pv, ntw, y). Mapping capacity: from (pv, sd, y) to (pv, ntw, y)
    Completed in 0.0002727508544921875 seconds
--- Aspect (capacity) not defined at ho, a variable will be created assuming y as the temporal index
--- Mapping capacity: from (pv, ho, y) to (pv, ntw, y)
    Completed in 0.0003218650817871094 seconds
--- Aspect (capacity) not defined at ny, a variable will be created assuming y as the temporal index
--- Mapping capacity: from (pv, ny, y) to (pv, ntw, y)
    Completed in 0.00019502639770507812 seconds
--- Creating map to (usd, ntw, y, capacity, pv). Mapping spend: from (usd, ho, y, capacity, pv) to (usd, ntw, y, capacity, pv)
    Completed in 0.0002887248992919922 seconds
--- Mapping spend: from (usd, sd, y, capacity, pv) to (usd, ntw, y, capacity, pv)
    Completed in 0.0002338886260986328 seconds
--- Mapping spend: from (usd, ny, y, capacity, pv) to (us

## Linkages 

A grid of valid linkages can be created from sources to sinks.

Linkages are always one directional, stating bi, created a link from source to sink and the other way round 

If multiple links exist between two locations with different distances, it is necessary to create named links. 
Again, it needs to be stated whether the links are bi directional 

In [ ]:
m.Link(source = m.ho, sink=m.sd, dist = 2000, bi = True)
# m.Link(source = m.sd, sink=m.ny, dist = 1500)
m.road = Link(source = m.ho, sink=m.ny, dist = 3000, bi = True)
# m.rail = Link(source = m.ho, sink=m.ny, dist = 5000, bi = True)
m.links 


[ho-sd, sd-ho, road, -road]

In [14]:
m.ho.links(m.sd)

ho is source and sd is sink in ho-sd
sd is source and ho is sink in sd-ho


[ho-sd, sd-ho]

## Transits

Declare some transport options between across the linkages 


In [ ]:
m.grid = Transport()
m.grid(m.power)
m.grid.capacity.x.forall([m.ho-m.sd, m.road]) <= 100
m.grid.operate.prep(norm = True).forall([m.ho-m.sd, m.road]) <= [[0.9, 0.8, 0.5, 0.7],
                                                           [0.8, 0.7, 0.6, 0.9]]
m.grid.operate[m.usd.spend].forall(m.links) == 2000
m.grid.capacity[m.usd.spend] == 1000  

aaaa  [x_capacity[4]]
aaaa  [x_capacity[5]]
aaaa  [capacity[10]]
aaaa  [capacity[10]]
aaaa  [capacity[10]]
aaaa  [capacity[10]]
aaaa  [capacity[11]]
aaaa  [capacity[11]]
aaaa  [capacity[11]]
aaaa  [capacity[11]]
--- Creating map to (grid, ho-sd, y). Mapping operate: from (grid, ho-sd, q) to (grid, ho-sd, y)
aaaa  [operate[34], operate[35]]
aaaa  [operate[34], operate[35], operate[36]]
aaaa  [operate[34], operate[35], operate[36], operate[37]]
aaaa  [operate[42]]
aaaa  [operate[43]]
--- Creating map to (grid, road, y). Mapping operate: from (grid, road, q) to (grid, road, y)
aaaa  [operate[38], operate[39]]
aaaa  [operate[38], operate[39], operate[40]]
aaaa  [operate[38], operate[39], operate[40], operate[41]]
aaaa  [operate[44]]
aaaa  [operate[45]]
--- Creating map to (usd, ntw, y, capacity, grid). Mapping spend: from (usd, ho, y, capacity, grid) to (usd, ntw, y, capacity, grid)
--- Creating map to (usd, ntw, y, capacity, grid). Mapping spend: from (usd, ho, y, capacity, grid) to (usd,

In [ ]:
m.grid.locate(m.ho-m.sd, m.road)

--- Assuming  grid capacity is unbounded in (sd-ho, y)
--- Assuming  grid capacity is unbounded in (-road, y)
--- General Resource Balance for power in (ho, q): adding ship_out from grid
aaaa  [release[0], ship_out[0]]
aaaa  [release[1], ship_out[1]]
aaaa  [release[2], ship_out[2]]
aaaa  [release[3], ship_out[3]]
--- General Resource Balance for power in (sd, q): adding ship_in from grid
aaaa  [release[4], ship_in[0]]
aaaa  [release[5], ship_in[1]]
aaaa  [release[6], ship_in[2]]
aaaa  [release[7], ship_in[3]]
aaaa  [operate[34], ship_out[0]]
aaaa  [operate[35], ship_out[1]]
aaaa  [operate[36], ship_out[2]]
aaaa  [operate[37], ship_out[3]]
aaaa  [operate[34], ship_in[0]]
aaaa  [operate[35], ship_in[1]]
aaaa  [operate[36], ship_in[2]]
aaaa  [operate[37], ship_in[3]]
--- General Resource Balance for power in (sd, q): adding ship_out from grid
aaaa  [release[4], ship_in[0], ship_out[4]]
aaaa  [release[5], ship_in[1], ship_out[5]]
aaaa  [release[6], ship_in[2], ship_out[6]]
aaaa  [release[7

In [17]:
m.show()

# Mathematical Program for example4

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
m.ho.operations(m.pv, m.wf, m.lii)
m.sd.operations(m.pv, m.wf, m.lii)
m.ny.operations(m.pv, m.wf, m.lii)

--- General Resource Balance for power in (ho, q): adding produce
aaaa  [release[0], ship_out[0], ship_in[4], produce[0]]
aaaa  [release[1], ship_out[1], ship_in[5], produce[1]]
aaaa  [release[2], ship_out[2], ship_in[6], produce[2]]
aaaa  [release[3], ship_out[3], ship_in[7], produce[3]]
aaaa  [operate[20]]
aaaa  [operate[21]]
aaaa  [operate[22]]
aaaa  [operate[23]]
aaaa  [operate[29]]
--- General Resource Balance for power in (ho, q): adding produce
aaaa  [release[0], ship_out[0], ship_in[4], produce[0], produce[4]]
aaaa  [release[1], ship_out[1], ship_in[5], produce[1], produce[5]]
aaaa  [release[2], ship_out[2], ship_in[6], produce[2], produce[6]]
aaaa  [release[3], ship_out[3], ship_in[7], produce[3], produce[7]]
aaaa  [operate[0]]
aaaa  [operate[1]]
aaaa  [operate[2]]
aaaa  [operate[3]]
aaaa  [operate[13]]
--- Assuming  power.lii inventory capacity is unbounded in (ho, y)
--- Creating map to (power.lii, ntw, y). Mapping invcapacity: from (power.lii, ho, y) to (power.lii, ntw, y)


In [19]:
m.show()

# Mathematical Program for example4

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.solar.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.usd.spend.opt()

bbbb spend (usd, ntw, y) False
--- Mapping spend: from (usd, capacity, wf, sd, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, wf, ho, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, wf, ny, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, wf, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, pv, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, pv, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, lii, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, lii, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, consume, cement, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, ho-sd, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, sd-ho, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, road, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, -road, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, grid, ntw, y) to (usd,

Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file example4.mps
Reading time = 0.00 seconds
EXAMPLE4: 233 rows, 241 columns, 572 nonzeros
--- Optimizing example4 using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 233 rows, 241 columns and 572 nonzeros
Model fingerprint: 0x30d71bce
Variable types: 234 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+06]
Presolve removed 166 rows and 170 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible
Best

In [ ]:
m.sol()

'Use .opt() to generate solution'

In [ ]:
m.export.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.spend.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>